In [6]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
import numpy as np
import pandas as pd


# Convert images to live image landmarks version:
Below takes in an array where each element is the file name of an image in the folder data/unlabled/. The script reads the image, runs a hand detection algorithm, then overlays the results and saves the output to the folder data/labeled/ with the original file name + "_marked.png"


TODO: Save the keypoints into a separate csv to create a training dataset

In [60]:
# For static images:
IMAGE_FILES = ["hands3.jpg"]
with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=2,
        min_detection_confidence=0.5) as hands:
  for idx, file_ori in enumerate(IMAGE_FILES):
    file = "./data/unlabeled/" + file_ori
    print("iteration",idx,"file,",file)
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).

    # image = cv2.flip(cv2.imread(file), 1)
    image = cv2.imread(file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert the BGR image to RGB before processing.
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      print(results.multi_hand_landmarks)
      for hand_landmarks in results.multi_hand_landmarks:
        # landmarks.append()
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    else:
      print("no landmarks found for",file)

    cv2.imwrite("./data/labeled/" +str(file_ori)[:-4]+ "marked.png",cv2.flip(image, 1))

iteration 0 file, ./data/unlabeled/hands3.jpg
[landmark {
  x: 0.5648515224456787
  y: 0.7065058946609497
  z: -1.6709475403331453e-06
}
landmark {
  x: 0.43671467900276184
  y: 0.6010831594467163
  z: -0.05960231274366379
}
landmark {
  x: 0.3417746424674988
  y: 0.495012104511261
  z: -0.07195031642913818
}
landmark {
  x: 0.3244509696960449
  y: 0.3697652220726013
  z: -0.0785597711801529
}
landmark {
  x: 0.40388110280036926
  y: 0.31218308210372925
  z: -0.062408093363046646
}
landmark {
  x: 0.4755207300186157
  y: 0.37364351749420166
  z: 0.009500401094555855
}
landmark {
  x: 0.4729347229003906
  y: 0.29737287759780884
  z: -0.050254132598638535
}
landmark {
  x: 0.44780904054641724
  y: 0.4066506624221802
  z: -0.07407911121845245
}
landmark {
  x: 0.4525124132633209
  y: 0.40707287192344666
  z: -0.08487633615732193
}
landmark {
  x: 0.5736780166625977
  y: 0.3985717296600342
  z: 0.01609756425023079
}
landmark {
  x: 0.565618097782135
  y: 0.3255450129508972
  z: -0.04090666

Now, we want the relative normalized coordinates of thes hand so it is invariant to location and image size!

In [3]:

# For static images:
IMAGE_FILES = ["hands.jpg"]
with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=2,
        min_detection_confidence=0.5) as hands:
  for idx, file_ori in enumerate(IMAGE_FILES):
    file = "./data/unlabeled/" + file_ori
    print("iteration",idx,"file,",file)
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).

    # image = cv2.flip(cv2.imread(file), 1)
    image = cv2.imread(file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert the BGR image to RGB before processing.
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # Extract the hand landmarks
    landmarks = []
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get the bounding box of the hand
            x_min, y_min, x_max, y_max = float('inf'), float('inf'), float('-inf'), float('-inf')
            for landmark in hand_landmarks.landmark:
                x, y = landmark.x, landmark.y
                x_min = min(x_min, x)
                y_min = min(y_min, y)
                x_max = max(x_max, x)
                y_max = max(y_max, y)
            
            # Normalize the coordinates with respect to the bounding box of the hand
            for landmark in hand_landmarks.landmark:
                x, y, z = landmark.x, landmark.y, landmark.z
                x_norm = (x - x_min) / (x_max - x_min)
                y_norm = (y - y_min) / (y_max - y_min)
                landmarks.append([x_norm, y_norm, z])

    # Convert the landmarks to a feature vector
    feature_vector = np.array(landmarks).flatten()

    # Print the feature vector
    print(feature_vector)

iteration 0 file, ./data/unlabeled/hands.jpg
[ 1.00000000e+00  4.34967835e-01 -3.47916576e-08  8.82582635e-01
  7.59323158e-01 -1.83860995e-02  6.89516073e-01  9.33540385e-01
 -2.44351812e-02  5.23791267e-01  9.92139709e-01 -3.15571316e-02
  3.82077427e-01  1.00000000e+00 -3.75392064e-02  5.04185301e-01
  6.06368761e-01  6.72204792e-03  3.23579050e-01  7.12554842e-01
 -1.75520517e-02  2.25997013e-01  8.39138381e-01 -4.18161079e-02
  1.51806618e-01  9.76945895e-01 -5.76517135e-02  4.77653627e-01
  4.08366269e-01  7.47234328e-04  2.59541127e-01  4.45197047e-01
 -1.72595009e-02  1.14985974e-01  5.40388545e-01 -3.80484127e-02
  0.00000000e+00  6.53343661e-01 -5.24306186e-02  4.98473873e-01
  2.32378382e-01 -1.09612327e-02  2.92032793e-01  2.19396905e-01
 -2.99397297e-02  1.55154329e-01  2.84264374e-01 -4.68715392e-02
  4.01836178e-02  3.75865428e-01 -5.75676374e-02  5.54889918e-01
  6.97741740e-02 -2.51207817e-02  3.97184439e-01  0.00000000e+00
 -4.07977998e-02  2.82935436e-01  1.45630962e

Next, we should use the above and create a csv training set where, for valid cases, we save the label (ex "A") as well as the extracted features, from the image using the above code, as a row in the file. When we want to use this file for training, we simply need to set the first column to the labels and the remaining columns to be the features. 

# Generate data set

In [8]:
# TO DO: MAke csv as described
train = pd.read_csv("./data/American Sign Language Letters.v1-v1.tensorflow/train/_annotations.csv")
test = pd.read_csv("./data/American Sign Language Letters.v1-v1.tensorflow/test/_annotations.csv")
valid = pd.read_csv("./data/American Sign Language Letters.v1-v1.tensorflow/valid/_annotations.csv")
train.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,U7_jpg.rf.0037faea78f8a89329a93006132921b3.jpg,390,390,U,38,69,347,389
1,P12_jpg.rf.0046c1c30abbbccd31716c5b2ad835b9.jpg,372,372,P,84,203,330,332
2,K4_jpg.rf.00821732715c9137b8060360770ea1d8.jpg,372,372,K,42,12,351,369
3,W6_jpg.rf.00d19bc3a49f6469e2afa3aa92f14ff4.jpg,412,412,W,22,73,377,412
4,J30_jpg.rf.00d20e595026b31773ded47509545471.jpg,382,382,J,122,204,250,334


In [42]:
def getFeatures(files,root,labels):
    """ 
    Reads in a set of image files and returns a list of the hand_features for each
    
    """
    features = [] # store features here
    labels_true = [] # We make sure that the corresponding label is there. Covers the case where no_hand is detected

    IMAGE_FILES = files
    with mp_hands.Hands(
            static_image_mode=True,
            max_num_hands=2,
            min_detection_confidence=0.5) as hands:
        for idx, file_ori in enumerate(IMAGE_FILES):
            file = root + file_ori # ex root: "./data/unlabeled/" 
            # print("iteration",idx,"file,",file)
            # Read an image, flip it around y-axis for correct handedness output (see
            # above).

            # image = cv2.flip(cv2.imread(file), 1)
            image = cv2.imread(file)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Convert the BGR image to RGB before processing.
            results = hands.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            # Extract the hand landmarks
            landmarks = []
            if results.multi_hand_landmarks:
                #Label update:
                labels_true.append(labels[idx])
                #Hand features updates
                for hand_landmarks in results.multi_hand_landmarks:
                    # Get the bounding box of the hand
                    x_min, y_min, x_max, y_max = float('inf'), float('inf'), float('-inf'), float('-inf')
                    for landmark in hand_landmarks.landmark:
                        x, y = landmark.x, landmark.y
                        x_min = min(x_min, x)
                        y_min = min(y_min, y)
                        x_max = max(x_max, x)
                        y_max = max(y_max, y)
                    
                    # Normalize the coordinates with respect to the bounding box of the hand
                    for landmark in hand_landmarks.landmark:
                        x, y, z = landmark.x, landmark.y, landmark.z
                        x_norm = (x - x_min) / (x_max - x_min)
                        y_norm = (y - y_min) / (y_max - y_min)
                        landmarks.append([x_norm, y_norm, z])

            # Convert the landmarks to a feature vector
            feature_vector = np.array(landmarks)
            features.append(feature_vector)
            # Print the feature vector
            # print(feature_vector)
    return features, labels_true
            
test_feat, labels = getFeatures(['hands.jpg','hands2.jpg'],"./data/unlabeled/",["A","B"])
test_feat = np.array(test_feat).reshape(len(test_feat,))
print(test_feat.shape)


(2,)


C:\Users\andre\AppData\Local\Temp\ipykernel_16056\2743454335.py:59: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_feat = np.array(test_feat)


In [20]:
# Sends list of file names, the source directory, and the labels list
x_train, y_train = getFeatures(train.iloc[:,0],"./data/American Sign Language Letters.v1-v1.tensorflow/train/", train.iloc[:,3].tolist())


In [27]:
df_train = pd.DataFrame(data=x_train)
df_train.to_csv("./data/American Sign Language Letters.v1-v1.tensorflow/train/train_features.csv")

df_train = pd.DataFrame(data=y_train)
df_train.to_csv("./data/American Sign Language Letters.v1-v1.tensorflow/train/train_labels.csv")


One hot encode letters

In [ ]:
def oneHot(letters):
    """ 
    One hot encodes letter labels
    """
    hot_labels = []
    for letter in letters:
        encoded = [0]*26
        #Check if invalid character
        if (ord(letter) < 65) or (ord(letter) > 122):
            encoded = [-1]*26
            print("Warning: Invalid character detected")
        #Otherwise valid and update encoded vec    
        else:
            if ord(letter) <96: #Upper case
                encoded[ord(letter)-65] = 1
            else: #Lowercase
                encoded[ord(letter)-97] = 1
        hot_labels.append(encoded)
    return hot_labels
## TEST
letters_unit_test = ["A", "a","Z","z","L"]
oneHot(letters_unit_test)

In [28]:
#One hot encode
y_train_hot = oneHot(y_train)

In [29]:
#model 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


